In [2]:
from sklearn.model_selection import GridSearchCV
import torch
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms
import torchvision.models as models
import torch.nn as nn
import time
from sklearn.metrics import balanced_accuracy_score

import numpy as np
import pandas as pd

/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train = torchvision.datasets.ImageFolder(root=r"/work/-20230124-132407/data", transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(train, batch_size=64, shuffle=True, num_workers=4)

test = torchvision.datasets.ImageFolder(root=r"/work/-20230124-132407/data", transform=transforms.ToTensor())
test_loader = torch.utils.data.DataLoader(test, batch_size=64, shuffle=True, num_workers=4)

## Equilibrage des données d'entrainement
permet juste de rééquilibrer les classes en supprimant un sous échantillon aléatoire de données dans la classe majoritaire. De manière à ce que les 2 classes aient à peu près les mêmes effectifs

In [ ]:
# récupérer les images du train_dataset
images = []
labels = []
for image, label in train:
    images.append(image)
    labels.append(label)
print(type(images))

<class 'list'>


In [ ]:
print(len(labels))

2880


In [ ]:
# on  vérifie le déséquilibre des classes
print(sum(np.array(labels)==1)/len(labels))
print(sum(np.array(labels)!=1)/len(labels))

0.121875
0.878125


In [ ]:
# indice de tous les 1
id_train_1 = np.where((np.array(labels)==1))

# on copie notre dataset original pour ensuite supprimer toutes les data classées en 1
y_train_0 = labels.copy()
X_train_0 = images.copy()

# on supprimer tous les 1 donc toutes les data classées en 0
for i in sorted(id_train_1[0], reverse = True):
    del y_train_0[i]
    del X_train_0[i]

# indice de tous les 0
id_train_0 = np.where((np.array(labels)==0))

# on copie notre dataset original pour ensuite supprimer toutes les data classées en 0
y_train_1 = labels.copy()
X_train_1 = images.copy()

# on supprimer tous les 0 donc toutes les data classées en 1
for i in sorted(id_train_0[0], reverse = True):
    del y_train_1[i]
    del X_train_1[i]

In [ ]:
print(len(y_train_1))
print(len(X_train_1))

351
351


In [ ]:
# on supprime un sous échantillon aléatoire de la classe majoritaire qui compte 90% des éléments de cette classe
import random as rd

X_new_train_0 = X_train_0.copy()
y_new_train_0 = y_train_0.copy()

list_index = [k for k in range(len(X_train_0))]

for i in sorted(rd.sample(list_index,int(len(X_train_0)*0.85)), reverse=True):
    del X_new_train_0[i]
    del y_new_train_0[i]


In [ ]:
print(len(X_new_train_0))
print(len(y_new_train_0))

380
380


In [ ]:
X_new_train = X_new_train_0 + X_train_1
y_new_train = y_new_train_0 + y_train_1

In [ ]:
# on  vérifie le réquilibrage des classes
print(sum(np.array(y_new_train)==1)/len(y_new_train))
print(sum(np.array(y_new_train)!=1)/len(y_new_train))

0.4801641586867305
0.5198358413132695


In [ ]:
new_inputs = torch.stack([t[0] for t in X_new_train])
new_labels = torch.tensor(y_new_train)
print("Il reste",len(new_labels),"données !")

Il reste 731 données !


In [ ]:
new_train = torch.utils.data.TensorDataset(new_inputs, new_labels)

# Boucle d'entrainement et d'évaluation des performances du modèle

Pour le moment, nous avons enlever juste le dernier neurone.
En fonction des résultats obtenus nous devrons probablement enlever quelques couches vers la fin.

## Fonction d'entrainement du modèle

In [5]:
def train(model, train_loader, optimizer, criterion):
    """
    Cette fonction permet d'entrainer un réseau de neuronne :
        -model : le réseau à entrainer
        -train_loader : le set d'entrainement (issus de torch.utils.data.DataLoader)
        -optimizer : choix de l'optimiseur
        -criterion : choix de la fonction de coût
    """
    model.train()
    for inputs, labels in train_loader:
        #réinitialisation du gradient
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

## Fonction pour évaluer les performance d'un modèle

In [6]:
def validate(model, data, criterion):
    """
    Cette fonction permet de calculer la précision pour l'ensemble
    des classes prédites d'un modèle déja entrainé:
        -model : modèle pré-entrainé
        -data : set de donnée validation ou entrainement
        -criterion : la fonction de coût utilisé lors de l'apprentissage
    """
    model.eval()
    
    #Evaluation des vrais positifs et vrais négatifs
    nb_correct_predictions = [0] * 2 #2 classes
    nb_total_predictions = [0] * 2 #2 classes
    
    y_true = []
    y_pred = []
    
    with torch.no_grad():
        for inputs, labels in data:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            
            y_true += labels.tolist()
            y_pred += predicted.tolist()
            
            balanced_acc = balanced_accuracy_score(y_true, y_pred)
            
            for i in range(len(labels)):
                if labels[i] == predicted[i]:
                    nb_correct_predictions[labels[i]] += 1
                nb_total_predictions[labels[i]] += 1
                
    good_pred = [nb_correct_predictions[i] / nb_total_predictions[i] for i in range(len(data.dataset.classes))]
    
    return good_pred, balanced_acc

## Entrainement et évaluation des perfomances du modèle 

In [7]:
def perf_evaluation(model, data_train, data_val, num_epochs,
                    early_stopping_threshold,
                    criterion, optimizer, scheduler):
    
    """
    Cette fonction permet d'entrainer un modèle.
    La fonction renvoi le meilleurs modèle
    avec les statistiques de perofrmance réseauen fonction de l'epoch
    
        -model : modèle a entrainer
        -data_train : set d'apprentissage
        -data_val : set de validation
        
        -num_epochs : le nombre d'epoques
        -early_stopping_threshold : le nombre d'époque minimum
            ou l'on n'observe aucune amèliariation des performances
        -criterion : choix de la fonction de coût
        -optimizer : choix de l'optimiseur
        scheduler : 
    """

    counter = 0
    best_model_wts = model.state_dict()
    best_acc = 0.0
    
    bal_acc_train = []
    bal_acc_val = []
    
    true_neg_train = []
    true_pos_train = []
    
    true_neg_val = []
    true_pos_val = []
    
    time_list = []
    epoch_list = []
    
    for epoch in range(num_epochs):
        
        #Entrainement du modèle et évaluation du temps
        #d'entrainement
        start = time.time()
        train(model, data_train, optimizer, criterion)
        end = time.time()
        
        #Performances sur le set d'entrainement
        perf_train = validate(model, data_train, criterion)
        print("Train set. True negative : ", perf_train[0][0],
                "true positive", perf_train[0][1],
                "balanced accuracy", perf_train[1])
        
        #Performances sur le set de validation
        perf_val = validate(model, data_val, criterion)
        print("validation set. True negative : ", perf_val[0][0],
                "true positive", perf_val[0][1],
                "balanced accuracy", perf_val[1])
        
        bal_acc_train.append(perf_train[1])
        bal_acc_val.append(perf_val[1])
        
        true_neg_train.append(perf_train[0][0])
        true_pos_train.append(perf_train[0][1])
        
        true_neg_val.append(perf_val[0][0])
        true_pos_val.append(perf_val[0][1])
        
        time_list.append(round((end-start) * 10**3))
        epoch_list.append(epoch)
        
        
        if perf_val[1] > best_acc:
            best_acc = perf_val[1]
            best_model_wts = model.state_dict()
            counter = 0
        else:
            counter += 1

        if counter >= early_stopping_threshold:
            print("Early stopping at epoch: ", epoch+1)
            break
    
    perf = pd.DataFrame({'balanced_acc_train' : bal_acc_train,
                         'balanced_acc_val' : bal_acc_val,
                         'true_negative_train' : true_neg_train,
                         'true_positive_train' : true_pos_train,
                         'true_negative_val' : true_neg_val,
                         'true_positive_val' : true_pos_val,
                         'epochs' : epoch_list,
                         'time_ms': time_list})
    
    return best_model_wts, perf

# Liste des modèles

Plusieurs architectures de réseaux sont possibles pour avoir plusieurs canaux :

- modèle classique : pas de modifications. Afin d'utiliser ces derniers pour faire de la classification sur n versions d'une image, il faut alors les concaténer, ce qui est relativement simple.

- modèle à n canaux d'entrée : on modifie la première couche de convolution de sorte à cette dernière ait autant de canaux d'entrée qu'il y a de version d'une même image. Mais il faudra alors créer une nouvelle classe et de nouvelles fonctions avec les attributs du data loader afin d'entraîner et d'utiliser ce neurone.

- modèle à n réseaux réunis en un seul neurone de sortie : cette architecture vise à utiliser autant de réseaux qu'il y a de version d'une même image et lier l'ensemble des réseaux à un seul neurone de sortie. Attention cette architecture présente de nombreux désavantages (temps d'apprentissage et d'optimisation). D'autant plus que si nous entraîner les dernières couches du réseau le nombre de combinaisons possibles et le temps d'entraînement risque d'exploser. En plus de cela, il faudra également créer une nouvelle classe de data loader. Cela revient presque à avoir un système avec "vote"

In [8]:
mod = models.resnet18(pretrained=True)
num_ftrs = mod.fc.in_features
mod.fc = nn.Linear(num_ftrs, 2)

mod_green = models.resnet18(pretrained=True)
num_ftrs = mod_green.fc.in_features
mod_green.fc = nn.Linear(num_ftrs, 2)

mod_skelet = models.resnet18(pretrained=True)
num_ftrs = mod_skelet.fc.in_features
mod_skelet.fc = nn.Linear(num_ftrs, 2)

/shared-libs/python3.9/py/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/shared-libs/python3.9/py/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
#mod_n_chanels = models.resnet18(pretrained=True)

# Récupérer la première couche de convolution
#conv1 = mod_n_chanels.conv1

# paramètres de la nouvelle couche à n cannaux
#in_channels = 3
#out_channels = conv1.out_channels
#kernel_size = conv1.kernel_size
#stride = conv1.stride
#padding = conv1.padding

#Construction de la couche à n cannaux
#conv_n_channels = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
#conv_n_channels.weight.data.copy_(conv1.weight.data)

#mod_n_chanels.conv1 = conv_n_channels

# Réinitialiser la dernière couche pour adapter au nombre de classes
#num_ftrs = mod_n_chanels.fc.in_features
#mod_n_chanels.fc = nn.Linear(num_ftrs, 2)

# Analyse des modèles

In [10]:
num_epochs = 200
early_stopping_threshold = 20
learning_rate = 0.1

criterion = nn.CrossEntropyLoss()
optimizer = Adam(mod.parameters())
scheduler = StepLR(optimizer, step_size=10)

In [11]:
best_model_wts, perf = perf_evaluation(mod, train_loader, test_loader, num_epochs,
                                       early_stopping_threshold,
                                       criterion, optimizer, scheduler)

KeyboardInterrupt: 

In [ ]:
#Sauvegarde du modèle
torch.save(best_model_wts.state_dict(),'/work/Models_history/mod1.pth')

In [ ]:
perf

# Optimisation des paramètres

In [ ]:
param_grid = {'step_size': [1, 5, 7, 10, 20, 30, 50, 100],
              'gamma': [0.001, 0.01, 0.1, 0.5, 1, 10]}

In [ ]:
model = models.resnet18()

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

model.load_state_dict(best_model_wts)

criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters())

In [ ]:
grid = GridSearchCV(model, param_grid={'step_size': step_size_range,
                                       'gamma': gamma_range}, cv=5)

In [ ]:
grid.fit(train_loader)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6f211ba4-8439-4b4f-a72d-48759d094385' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>